In [1]:
%matplotlib widget

import os
import sys
sys.path.insert(0,'../../src')

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from process_results import *
from dataset_tester import *
from get_landmarks import GetLandmarks 
#from LandmarkTester_alt import LandmarkTester

from landmark_utils import vis_landmarks, compare_landmarks
from vis_utils import *


import cv2
import numpy as np
import time
import timeit
from pathlib import Path
import csv
import pickle

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from scipy import linalg
import seaborn as sns

In [2]:
MEEI_path = Path('/media/2TB/data/bp_pat_datasets/MEEI_Standard_Set')
pat_info = pd.read_excel(MEEI_path/"pat_info.xlsx", sheet_name='Sheet1')

In [3]:
MEEI_pat_info = pd.read_pickle('../../src/pickles/MEEI_pat_info.pkl')
all_pat_info = '../../src/pickles/all_pats_processed_data.pickle'

if os.path.getsize(all_pat_info) > 0:
    with open(all_pat_info,'rb') as f:
        unpickler = pickle.Unpickler(f)
        loaded_pats = unpickler.load(f)
        print(loaded_pats)
else:
    print('The file is empty.')

The file is empty.


In [5]:
pat_info

,Category,Sub-category,#,Side,Gender,Age
0,Flaccid,Complete,1,Left,Male,54
1,Flaccid,Complete,2,Right,Female,16
2,Flaccid,Complete,3,Right,Male,38
3,Flaccid,Complete,4,Left,Female,31
4,Flaccid,Complete,5,Left,Female,52
5,Flaccid,Mild,1,Left,Male,72
6,Flaccid,Mild,2,Right,Female,55
7,Flaccid,Mild,3,Left,Male,75
8,Flaccid,Moderate,1,Left,Male,51
9,Flaccid,Moderate,3,Left,Female,62


In [7]:
# x - choose one
all_ids = loaded_pats[0]
all_imgs = loaded_pats[1]
all_lms = loaded_pats[2]
all_features = loaded_pats[3]

# y
all_hb = loaded_pats[4]

NameError: name 'loaded_pats' is not defined

In [8]:
# Find NaN and Inf values
bad_indices = np.where(np.isnan(all_features).sum(axis=1) > 0)[0]
bad_indices = np.append(bad_indices, np.where(np.isinf(all_features).sum(axis=1) > 0)[0])
bad_indices.sort()
bad_indices

NameError: name 'all_features' is not defined

In [9]:
# drop bad indices 
# x
all_ids = np.delete(all_ids, bad_indices, axis=0)
all_imgs = np.delete(all_imgs, bad_indices, axis=0)
all_lms = np.delete(all_lms, bad_indices, axis=0)
all_features = np.delete(all_features, bad_indices, axis=0)

# y
all_hb = np.delete(all_hb, bad_indices, axis=0)

all_features.shape, all_hb.shape

NameError: name 'all_ids' is not defined

In [10]:
plot_df = pd.DataFrame(all_features, columns = ['f'+str(i) for i in range(0, 29)])

NameError: name 'all_features' is not defined

In [11]:
plot_df['hb'] = all_hb

NameError: name 'all_hb' is not defined

In [12]:
def feature_plot(df, which_feature, group_by):
    sns.displot(df, x=which_feature, hue=group_by, element="step")
    

In [13]:
for i in range(0, 29):
    feature_plot(plot_df, 'f'+str(i), 'hb')

NameError: name 'plot_df' is not defined

In [14]:
# Split IDs into train and test
random.seed(85)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 42)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

Training IDs: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 19 20 21 22 24 27 29 30 31
 32 33 34 35 36 37 38 39 40 41 42 43 44 46 47 48 49 50]
Testing IDs: [ 1 16 17 18 23 25 26 28 45]


In [15]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train, x_test = all_features[train_idx], all_features[test_idx]
y_train, y_test = all_hb[train_idx], all_hb[test_idx]

print('Training x shape:', x_train.shape)
print('Training y shape:', y_train.shape)
print('Testing x shape:', x_test.shape)
print('Testing y shape:', y_test.shape)

NameError: name 'all_ids' is not defined

In [16]:
x_train[0], y_train[0]

NameError: name 'x_train' is not defined

In [17]:
x_test[0], y_test[0]

NameError: name 'x_test' is not defined

In [18]:
data_dmatrix = xgb.DMatrix(data=all_features,label=all_hb)

NameError: name 'all_features' is not defined

In [19]:
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 8, alpha = 10, n_estimators = 30)

In [20]:
xg_reg = xgb.XGBClassifier(
    tree_method="gpu_hist", enable_categorical=True, n_estimators=30, max_depth=10)

In [21]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

NameError: name 'y_train' is not defined

In [22]:
xg_reg.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [23]:
y_pred = xg_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %2f" % (rmse))
print('Accuracy:', round(accuracy_score(y_test, y_pred), 3))

NameError: name 'x_test' is not defined

In [24]:
y_pred[10:20], y_test[10:20]

NameError: name 'y_pred' is not defined

In [ ]:
# Now do data whitening
x = all_features
print('x.shape:', x.shape)

In [ ]:
# Center data
# By subtracting mean for each feature
xc = x.T - np.mean(x.T, axis=0)
xc = xc.T
print('xc.shape:', xc.shape, '\n')

In [ ]:
# Calculate covariance matrix
xcov = np.cov(xc, rowvar=True, bias=True)
print('Covariance matrix: \n', xcov, '\n')

In [ ]:
# Calculate Eigenvalues and Eigenvectors
w, v = linalg.eig(xcov)
# Note: Use w.real.round(4) to (1) remove 'j' notation to real, (2) round to '4' significant digits
print("Eigenvalues:\n", w.real.round(4), '\n')
print("Eigenvectors:\n", v, '\n')

In [ ]:
# Calculate inverse square root of Eigenvalues
# Optional: Add '.1e5' to avoid division errors if needed
# Create a diagonal matrix
diagw = np.diag(1/(w**0.5)) # or np.diag(1/((w+.1e-5)**0.5))
diagw = diagw.real.round(4) #convert to real and round off
print("Diagonal matrix for inverse square root of Eigenvalues:\n", diagw, '\n')

In [ ]:
# Calculate Rotation (optional)
# Note: To see how data can be rotated
xrot = np.dot(v, xc)
 
# Whitening transform using PCA (Principal Component Analysis)
wpca = np.dot(np.dot(diagw, v.T), xc)
 
# Whitening transform using ZCA (Zero Component Analysis)
wzca = np.dot(np.dot(np.dot(v, diagw), v.T), xc)

In [ ]:
# Overlay plot
plt.figure()
plt.scatter(xc[0,:], xc[1,:], label='centered', alpha=0.15)
plt.scatter(wpca[0,:], wpca[1,:], marker='+',label='wpca', alpha=0.5)
plt.scatter(wzca[0,:], wzca[1,:], marker='x', label='wzca', alpha=0.5)
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.legend()
 
plt.tight_layout()
plt.show()

In [ ]:
wzca.shape

In [ ]:
# We try again with whitened data
# x - choose one
processed_features = wzca
# y
#all_hb 

In [ ]:
# Split IDs into train and test
random.seed(85)
subject_ids = [i for i in range(51)]
train_ids = np.sort(np.array(random.sample(subject_ids, 42)))
test_ids = np.sort(np.array([i for i in subject_ids if i not in train_ids]))

print('Training IDs:',train_ids)
print('Testing IDs:',test_ids)

In [ ]:
# Next, match the IDs to the indices in the data
train_idx = np.sort(np.where(np.isin(all_ids, train_ids))[0])
test_idx = np.sort(np.where(np.isin(all_ids, test_ids))[0])

x_train2, x_test2 = processed_features[train_idx], processed_features[test_idx]
y_train2, y_test2 = all_hb[train_idx], all_hb[test_idx]

print('Training x shape:', x_train2.shape)
print('Training y shape:', y_train2.shape)
print('Testing x shape:', x_test2.shape)
print('Testing y shape:', y_test2.shape)

In [ ]:
x_train2[0], y_train2[0]

In [ ]:
x_test2[0], y_test2[0]

In [ ]:
#data_dmatrix = xgb.DMatrix(data=all_features,label=all_hb)

In [ ]:
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 8, alpha = 10, n_estimators = 30)

In [ ]:
xg_reg = xgb.XGBClassifier(
    tree_method="gpu_hist", enable_categorical=True, n_estimators=30, max_depth=10)


In [ ]:
le = LabelEncoder()
y_train2 = le.fit_transform(y_train2)

In [ ]:
x_train2.shape, y_train2.shape

In [ ]:
x_train2[10], y_train2[10]

In [ ]:
xg_reg.fit(x_train2, y_train2)
y_pred2 = xg_reg.predict(x_test2)
rmse2 = np.sqrt(mean_squared_error(y_test2, y_pred2))
print("RMSE: %2f" % (rmse2))
print('Accuracy:', round(accuracy_score(y_test2, y_pred2), 3))

In [ ]:
y_pred2[10:20], y_test2[10:20]